In [ ]:
#pip install line_profiler

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
import sys
from IPython.display import clear_output
import heapq
import pandas as pd

from google.colab import drive
import os

#from line_profiler import LineProfiler

In [ ]:
# Function to read the .turb files

def read_turb_file(filename):
  nodes = []
  with open(filename, 'r') as file:
      for line in file:
          # Split each line into x, y, and power
          x, y, power = map(float, line.split())

          # Store the data in a list
          nodes.append([x,y,power])

  return nodes

# Function to read the .cable files

def read_cables_file(filename):
  cables = []
  with open(filename, 'r') as file:

      cable_type = 1

      for line in file:

          # Split each line into x, y, and power
          capacity, price, availability = map(float, line.split())

          capacity = int(capacity)
          availability = int(availability)

          # Store the data in a list
          cables.append([cable_type,capacity,price,availability])

          cable_type += 1

  return cables


# Function to create a distance matrix

def create_dist_matrix(nodes):
  n = len(nodes)
  A = np.zeros((n,n))
  for i in range(n):
    for j in range(n):
      if nodes[i]!=nodes[j] and A[i][j]==0:
        A[i][j] =  math.sqrt( (nodes[j][0] - nodes[i][0])**2 + (nodes[j][1] - nodes[i][1])**2 )
        A[j][i] = A[i][j]

  return A

# Function to create a list contaning the angles with respect to substation

def compute_angles(nodes):
  n = len(nodes)
  angles = []
  angles.append(0.00)

  for i in range(1,n):
    # Use unit vector pointing rightwards as reference
    ax = 1.0
    ay = 0.0
    # Create vector b from substation to a given node
    bx = nodes[i][0] - nodes[0][0]
    by = nodes[i][1] - nodes[0][1]
    # Calculate the angle using the arctan2 function
    angle_radians = math.atan2(by, bx) - math.atan2(ay, ax)
    # Normalize the angle to the range [0, 2*pi]
    angle = angle_radians % (2 * math.pi)
    angles.append(angle)
  return angles

# Function to group together nodes

def node_grouping(nodes,indexed_angles,starting_turb,TPG):

  # Shift the list order based on the chosen starting node
  shifted_angles = []
  counter = 0
  n = len(indexed_angles)
  # Find the list position of the starting node
  for k in range(n):
    if indexed_angles[k][0] == starting_turb:
      i = k
  while counter < n:
    shifted_angles.append(indexed_angles[i])
    i = (i + 1) % n
    counter += 1
  indexed_angles = shifted_angles
  # Create the turbine groups
  groups = []
  while indexed_angles:
    group = []
    for i in range(TPG):
      if indexed_angles:
        item = indexed_angles.pop(0)
        node = item[0]
        group.append(node)
    groups.append(group)
  return groups


# Function to set the costs between nodes of different groups to infinity

def segregate(dist_matrix, groups):

  cost_matrix = np.copy(dist_matrix)
  n = cost_matrix.shape[0]
  g = len(groups)
  for i in range(1,n):
    for j in range(g):
      if i not in groups[j]:
        for k in groups[j]:
          cost_matrix[i][k] = float('inf')
  return cost_matrix


# Prim's algorithm to compute the MST
"""
def prim(A):
  n = A.shape[0]
  # Define an array B to hold the final adjacency matrix
  B = np.zeros((n,n))
  # Define the initial cost of the MST
  cost = 0.0
  # Define the set of nodes in the tree and its complement
  T = [0]
  Tc = []
  for i in range(1,n):
    Tc.append(i)
  # Add the node with minimum weight that links to the tree
  while Tc:
    min_weight = float('inf')
    for i in T:
      for j in Tc:
        if A[i][j] < min_weight:
          node_index = i
          neighbor_index = j
          min_weight = A[i][j]
    T.append(neighbor_index)
    Tc.remove(neighbor_index)
    B[node_index][neighbor_index] = 1
    B[neighbor_index][node_index] = 1
    cost = cost + min_weight

  return B
"""

def prim(A):
    n = A.shape[0]
    B = np.zeros((n,n))
    # Initialize the MST edges and total cost
    edges = []
    cost = 0.0
    # Initialize the set of nodes in the tree and its complement
    in_tree = {0}
    not_in_tree = set(range(1, n))
    # Add the neighbors of the first node to the priority queue
    for j in range(1, n):
        heapq.heappush(edges, (A[0][j], 0, j))

    while edges:
        if len(in_tree) == n:
          break
        # Get the minimum edge from the priority queue
        weight, node_index, neighbor_index = heapq.heappop(edges)
        # If the neighbor is not in the tree, add it to the tree
        if neighbor_index in not_in_tree:
            in_tree.add(neighbor_index)
            not_in_tree.remove(neighbor_index)
            # Update the total cost
            cost += weight
            # Add the edge to the MST
            B[node_index][neighbor_index] = 1
            B[neighbor_index][node_index] = 1
            # Add the neighbors of the newly added node to the priority queue
            for j in not_in_tree:
                heapq.heappush(edges, (A[neighbor_index][j], neighbor_index, j))

    return B



# Modified implementation of Depth First Search to compute the flows

def DFS(A,v,visited,next,leaves,flows):

  visited[v]=True

  if (v==0):
    next[v]=0

  n = A.shape[0]

  neighbor_counter = 0

  total_flow = 0

  for neighbor in range(n):

    if A[v][neighbor]==1 and not visited[neighbor]:
      neighbor_counter += 1
      next[neighbor]=v
      edge = (neighbor,v)
      if edge not in flows:
        flows[edge] = DFS(A,neighbor,visited,next,leaves,flows)
      total_flow += flows[edge]

  if neighbor_counter == 0:
    leaves.append(v)
    edge = (v,next[v])
    flows[edge] = 1
    return 1

  if (v == 0):
    return total_flow
  else:
    return 1 + total_flow

# Function to compute the prices associated with a choice of cable types

def compute_price(distance_matrix,flows,cable_types):
  info_dict = {}
  total_price = 0.0
  n_types = len(cable_types)

  # Compute total price based on cost function
  for (i,j),f in flows.items():
    # Use bool variable to keep track of overflow
    overflow = True
    # Select appropriate cable type
    for cable_type in cable_types:
      if cable_type[1]>=f:
        overflow = False
        info_dict[(i,j)] = (cable_type[0],overflow)
        total_price += cable_type[2]*distance_matrix[i][j]
        break
    if overflow==True:
      info_dict[(i,j)]= (cable_types[n_types-1][0],overflow)
      total_price += cable_types[n_types-1][2]*distance_matrix[i][j]

  return info_dict, total_price


#%load_ext line_profiler

def multi_sweep(nodes,cable_types,instance_number):

  # Create the distance matrix
  dist_matrix = create_dist_matrix(nodes)

  # Determine the TPG range
  n = dist_matrix.shape[0]-1
  C = math.ceil(n/cable_types[len(cable_types)-1][1]) +1
  TPG_low = n//C
  TPG_high = cable_types[len(cable_types)-1][1]//1

  # Initialize candidate solution
  best_info_dict = {}
  best_total_price = float('inf')

  # Initialize run counter
  counter = 1
  max = (TPG_high-TPG_low + 1)*n*2

  # Compute angles
  angles = compute_angles(nodes)

  # Create a list of tuples containing the index of the node and its angle
  indexed_angles = [(index, angle) for index, angle in enumerate(angles)]
  indexed_angles.pop(0)
  cclock_indexed_angles = indexed_angles = sorted(indexed_angles, key=lambda x: x[1])
  clock_indexed_angles = indexed_angles = sorted(indexed_angles, key=lambda x: x[1], reverse=True)

  for TPG in range(TPG_low,TPG_high + 1):
    for starting_turb in range(1,n+1):
      for direction in range(0,1+1):
        # Print current status

        clear_output(wait=True)
        print("Instance:",instance_number,"/ 220")
        print("Run:",counter,"/",max)

        # Run instance of Sweep
        if direction == 0:
          groups = node_grouping(nodes,cclock_indexed_angles,starting_turb,TPG)
        if direction == 1:
          groups = node_grouping(nodes,clock_indexed_angles,starting_turb,TPG)

        # Compute flow, select cable types and determine the cost
        cost_matrix = segregate(dist_matrix,groups)
        T = prim(cost_matrix)
        visited = [False]*T.shape[0]
        next = [0]*T.shape[0]
        leaves = []
        flows = {}
        DFS(T,0,visited,next,leaves,flows)
        info_dict, total_price = compute_price(dist_matrix,flows,cable_types)
        if total_price < best_total_price:
          best_info_dict = info_dict.copy()
          best_total_price = total_price

        counter += 1

  return best_info_dict, best_total_price

# Function to plot the results

def plot_results(nodes,info_dict,output_path):

  # Plotting turbines

  for i, (x, y, power) in enumerate(nodes):
      plt.scatter(x, y, color='black')
      #plt.text(x + 0.1, y, f'{i}', fontsize=8, verticalalignment='center')

  # Plotting connections

  for (i, j), (cable_type, _) in info_dict.items():
      xi, yi, _ = nodes[i]
      xj, yj, _ = nodes[j]
      dx, dy = xj - xi, yj - yi
      if cable_type == 1:
        color = 'xkcd:sky blue'
      if cable_type == 2:
        color = 'xkcd:green'
      if cable_type == 3:
        color = 'xkcd:red'
      plt.arrow(xi, yi, dx, dy, head_width=0.1, head_length=0.1, fc = color, ec = color,length_includes_head=True)

  # Add legend for cable types

  plt.legend(handles=[
      plt.Line2D([0], [0], color='xkcd:sky blue', lw=2, label='Type 1'),
      plt.Line2D([0], [0], color='xkcd:green', lw=2, label='Type 2'),
      plt.Line2D([0], [0], color='xkcd:red', lw=2, label='Type 3')
  ])

  plt.axis('off')
  plt.grid(False)

  #plt.show()

  # Save the plot as a PNG file
  plt.savefig(output_path, bbox_inches='tight', pad_inches=0)



In [ ]:
"""
### TEST SECTION ###############################################################

filename_turb = "n120_s05_t04_w05.turb"
filename_cables = "n120_s05_t04_w05.cable"
output_path = '/content/plot.png'

instance_number = 1
nodes = read_turb_file(filename_turb)
cable_types = read_cables_file(filename_cables)

#%lprun -f multi_sweep multi_sweep(nodes,cable_types,instance_number)

info_dict, total_price = multi_sweep(nodes,cable_types,instance_number)
plot_results(nodes,info_dict,output_path)


################################################################################
"""




In [ ]:
# PART 1 - Obtain the results from the sweep heuristic

# Mount Google Drive
drive.mount('/content/drive',force_remount=False)

# Define the path to the folder in your Google Drive
testset_folder_path = '/content/drive/MyDrive/testset_220_instances_cminplus1'

# Get the list of filenames in the folder
filenames = os.listdir(testset_folder_path)
filenames.remove('README.pdf')
filenames.remove('README.md')

# Create a folder to hold the test results
#sweep_results_folder_path = '/content/drive/MyDrive/sweep_results'
sweep_results_folder_path = '/content/drive/MyDrive/sweep_results_faster'


# Check if the folder exists
if not os.path.exists(sweep_results_folder_path):
    # Create the folder if it doesn't exist
    os.makedirs(sweep_results_folder_path)

#------------------------------------------------------------------------------#
# For each pair of .turb and .cable files, get the instance names

testcase_names = []

for filename in filenames:
  if filename.endswith(".turb"):
    testcase_name = filename.rsplit(".", 1)[0] # Split by ".", take the 1st part
    testcase_names.append(testcase_name)

# Create folders for each test instance
instance_number = 1

for name in testcase_names:
  testcase_folder_path = sweep_results_folder_path + '/' + name

  if not os.path.exists(testcase_folder_path):
    # Create the folder if it doesn't exist
    os.makedirs(testcase_folder_path)

  # RUN THE SWEEP ALGORITHM FOR EACH INSTANCE

  filename_turb = testset_folder_path + '/' + name + '.turb'
  filename_cables = testset_folder_path + '/' + name + '.cable'

  nodes = read_turb_file(filename_turb)
  cable_types = read_cables_file(filename_cables)
  info_dict = {}
  info_dict, total_price = multi_sweep(nodes,cable_types,instance_number)

  # Save the dictionary

  file_path_dict = testcase_folder_path + '/' + name + '_info.txt'
  # Open the file in write mode
  with open(file_path_dict, 'w') as file:
      # Write each key-value pair to the file
      for key, value in info_dict.items():
          file.write(f'{key}: {value}\n')

  # Save the cost

  file_path_cost = testcase_folder_path + '/' + name + '_cost.txt'

  # Open the file in write mode
  with open(file_path_cost, 'w') as file:
      # Write the number to the file
      file.write(str(total_price))

  # Save the plot

  plot_results(nodes,info_dict, testcase_folder_path + '/' + name + '.png')

  # Clear the plot for the next one
  plt.clf()

  instance_number += 1













In [ ]:
#                             SKIP THIS PART!!!!

'''
# PART 2 - Obtain the best costs from Murilo's work

# Mount Google Drive
drive.mount('/content/drive',force_remount=False)

# Define the path to the folder in your Google Drive
testset_folder_path = '/content/drive/MyDrive/sweep_results'

# Create a folder to hold Murilo's best costs

murilo_folder_path = '/content/drive/MyDrive/PaperAppliedEnergy'
murilo_folder_path_VNS = '/content/drive/MyDrive/PaperAppliedEnergy/ResultadosDivulgacaoVNS'
murilo_folder_path_Arq = '/content/drive/MyDrive/PaperAppliedEnergy/ResultadosDivulgacaoArq'
murilo_folder_path_ArqMH = '/content/drive/MyDrive/PaperAppliedEnergy/ResultadosDivulgacaoArqMH'

murilo_best_path = '/content/drive/MyDrive/murilo_best'
murilo_best_path_VNS = '/content/drive/MyDrive/murilo_best/VNS'
murilo_best_path_Arq = '/content/drive/MyDrive/murilo_best/Arq'
murilo_best_path_ArqMH = '/content/drive/MyDrive/murilo_best/ArqMH'


# Create main folder
if not os.path.exists(murilo_best_path):
  os.makedirs(murilo_best_path)
# Create subfolders
if not os.path.exists(murilo_best_path_VNS):
  os.makedirs(murilo_best_path_VNS)
if not os.path.exists(murilo_best_path_Arq):
  os.makedirs(murilo_best_path_Arq)
if not os.path.exists(murilo_best_path_ArqMH):
  os.makedirs(murilo_best_path_ArqMH)



# Locate best cost for VNS in Murilo's results folder---------------------------
folder_names = os.listdir(murilo_folder_path_VNS)
counter = 0
for instance in folder_names:
  counter += 1
  clear_output(wait=True)
  print("VNS INSTANCES READ:", counter)
  final_path = murilo_best_path_VNS + '/' + instance
  if not os.path.exists(final_path):
    os.makedirs(final_path)
  #files = os.listdir(murilo_folder_path_VNS + '/' + instance)
  # Filter files ending with .txt, they contain the costs
  #txt_files = [file for file in files if file.endswith('.txt')]
  txt_files = [instance + '_Ex' + str(counter) + '.txt' for counter in range(1, 11) ]
  best_cost = float('inf')
  for f in txt_files:
    # Read the last line
    read_path = murilo_folder_path_VNS + '/' + instance + '/' + f

    with open(read_path, 'r', encoding='latin-1') as file:
        # Move the pointer to the end of the file
        file.seek(0, os.SEEK_END)
        file_size = file.tell()
        # Start reading backwards until a newline character is found or until the beginning of the file is reached
        current_position = file_size - 1
        while current_position >= 0:
            file.seek(current_position, os.SEEK_SET)
            if file.read(1) == '\n':
                break
            current_position -= 1
        # Move back one position to avoid reading the newline character
        file.seek(current_position + 1, os.SEEK_SET)
        last_line = file.readline().strip()  # Read the last line
        # Split the last line by ':'
        parts = last_line.split(':')
        # Get the last part (which should be the number)
        last_part = parts[-1].strip()
    cost = float(last_part)
    if cost < best_cost:
      best_cost = cost
  file_path = final_path + '/best_cost.txt'
  float_str = str(best_cost)

  with open(file_path, 'w') as file:
        file.write(float_str)
#-------------------------------------------------------------------------------

# Locate best cost for Arq in Murilo's results folder---------------------------
folder_names = os.listdir(murilo_folder_path_Arq)
counter = 0
for instance in folder_names:
  counter += 1
  clear_output(wait=True)
  print("Arq INSTANCES READ:", counter)
  final_path = murilo_best_path_Arq + '/' + instance
  if not os.path.exists(final_path):
    os.makedirs(final_path)
  txt_files = [instance + '_Ex' + str(counter) + '.txt' for counter in range(1, 11) ]
  best_cost = float('inf')
  for f in txt_files:
    # Read the last line
    read_path = murilo_folder_path_Arq + '/' + instance + '/' + f

    with open(read_path, 'r', encoding='latin-1') as file:
        # Move the pointer to the end of the file
        file.seek(0, os.SEEK_END)
        file_size = file.tell()
        # Start reading backwards until a newline character is found or until the beginning of the file is reached
        current_position = file_size - 1
        while current_position >= 0:
            file.seek(current_position, os.SEEK_SET)
            if file.read(1) == '\n':
                break
            current_position -= 1
        # Move back one position to avoid reading the newline character
        file.seek(current_position + 1, os.SEEK_SET)
        last_line = file.readline().strip()  # Read the last line
        # Split the last line by ':'
        parts = last_line.split(':')
        # Get the last part (which should be the number)
        last_part = parts[-1].strip()
    cost = float(last_part)
    if cost < best_cost:
      best_cost = cost
  file_path = final_path + '/best_cost.txt'
  float_str = str(best_cost)

  with open(file_path, 'w') as file:
        file.write(float_str)
#-------------------------------------------------------------------------------


# Locate best cost for ArqMH in Murilo's results folder-------------------------
folder_names = os.listdir(murilo_folder_path_ArqMH)
counter = 0
for instance in folder_names:
  counter += 1
  clear_output(wait=True)
  print("ArqMH INSTANCES READ:", counter)
  final_path = murilo_best_path_ArqMH + '/' + instance
  if not os.path.exists(final_path):
    os.makedirs(final_path)
  txt_files = [instance + '_Ex' + str(counter) + '.txt' for counter in range(1, 11) ]
  best_cost = float('inf')
  for f in txt_files:
    # Read the last line
    read_path = murilo_folder_path_ArqMH + '/' + instance + '/' + f

    with open(read_path, 'r', encoding='latin-1') as file:
        # Move the pointer to the end of the file
        file.seek(0, os.SEEK_END)
        file_size = file.tell()
        # Start reading backwards until a newline character is found or until the beginning of the file is reached
        current_position = file_size - 1
        while current_position >= 0:
            file.seek(current_position, os.SEEK_SET)
            if file.read(1) == '\n':
                break
            current_position -= 1
        # Move back one position to avoid reading the newline character
        file.seek(current_position + 1, os.SEEK_SET)
        last_line = file.readline().strip()  # Read the last line
        # Split the last line by ':'
        parts = last_line.split(':')
        # Get the last part (which should be the number)
        last_part = parts[-1].strip()
    cost = float(last_part)
    if cost < best_cost:
      best_cost = cost
  file_path = final_path + '/best_cost.txt'
  float_str = str(best_cost)

  with open(file_path, 'w') as file:
        file.write(float_str)
#-------------------------------------------------------------------------------





In [ ]:
import re
import statistics as stat
import csv

# Function to order instances according to numbers

def extract_numbers(s):
    return [int(num) for num in re.findall(r'\d+', s)]

# Function to open the .txt files in read mode
def extract_cost(path):
  with open(path, 'r') as file:

      # Read the content of the file
      content = file.read().strip()

      # Convert the content to a float
      float_value = float(content)

  return float_value

def calculate_gap(obtained,best):
  return ((obtained - best)/best)*100

# PART 3 - Compare costs with Murilo's work-------------------------------------

# Mount Google Drive
drive.mount('/content/drive',force_remount=False)

# Define the path to the folder in your Google Drive
testset_folder_path = '/content/drive/MyDrive/sweep_results'

# Locate the folders with Murilo's best costs

murilo_best_path_VNS = '/content/drive/MyDrive/murilo_best/VNS'
murilo_best_path_Arq = '/content/drive/MyDrive/murilo_best/Arq'
murilo_best_path_ArqMH = '/content/drive/MyDrive/murilo_best/ArqMH'

# Create a list with the names of the 200 instances used in Murilo's results

instances = os.listdir(murilo_best_path_VNS)
instances = sorted(instances,key=extract_numbers)

# Compute the gap and the average gap and place it in a table
counter = 1
instance_dict = {}
VNS_gap_avg = []
Arq_gap_avg = []
ArqMH_gap_avg = []
for instance in instances:

  sweep_path = testset_folder_path + '/' + instance + '/' + instance + '_cost.txt'
  VNS_path = murilo_best_path_VNS + '/' + instance + '/' + 'best_cost.txt'
  Arq_path = murilo_best_path_Arq + '/' + instance + '/' + 'best_cost.txt'
  ArqMH_path = murilo_best_path_ArqMH + '/' + instance + '/' + 'best_cost.txt'

  sweep_cost = extract_cost(sweep_path)
  VNS_cost = extract_cost(VNS_path)
  Arq_cost = extract_cost(Arq_path)
  ArqMH_cost = extract_cost(ArqMH_path)

  VNS_gap = calculate_gap(sweep_cost,VNS_cost)
  Arq_gap = calculate_gap(sweep_cost,Arq_cost)
  ArqMH_gap = calculate_gap(sweep_cost,ArqMH_cost)

  VNS_gap_avg.append(VNS_gap)
  Arq_gap_avg.append(Arq_gap)
  ArqMH_gap_avg.append(ArqMH_gap)

  instance_dict[instance] = (VNS_gap,Arq_gap,ArqMH_gap)

  print("Instance:",counter,'/200')
  #print(instance_dict[instance])
  #clear_output(wait=True)
  counter += 1

VNS_mean = stat.mean(VNS_gap_avg)
Arq_mean = stat.mean(Arq_gap_avg)
ArqMH_mean = stat.mean(ArqMH_gap_avg)

csv_folder_path = testset_folder_path + '/results_table.csv'
# Write dictionary to CSV file
with open(csv_folder_path, 'w', newline='') as csvfile:
    fieldnames = ['Line', 'VNS gap', 'Arq gap', 'ArqMH gap']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for key, value in instance_dict.items():
        writer.writerow({'Line': key, 'VNS gap': value[0], 'Arq gap': value[1], 'ArqMH gap': value[2]})

    # Write averages
    writer.writerow({'Line': 'Average gap', 'VNS gap': VNS_mean, 'Arq gap': Arq_mean, 'ArqMH gap': ArqMH_mean})

print("CSV file has been created successfully.")